In [1]:
import os
import pandas as pd
import numpy as np
import collections
import tensorflow as tf
import pickle

In [2]:
number_of_crossvalidation_run=1
models_folder_name_test = os.path.join(os.getcwd(),'models','test',str(number_of_crossvalidation_run))
models_folder_name_train = os.path.join(os.getcwd(),'models','train',str(number_of_crossvalidation_run))
summaries_folder_name = os.path.join(os.getcwd(),'summaries','test', str(number_of_crossvalidation_run))

path_to_preprocessed_texts_test = os.path.join(models_folder_name_test,'recipes_test_dataset.pkl')
path_to_preprocessed_texts_train = os.path.join(models_folder_name_train,'recipes_train_dataset.pkl')

df_preprocessed_texts_test = pd.read_pickle(path_to_preprocessed_texts_test)
df_preprocessed_texts_train = pd.read_pickle(path_to_preprocessed_texts_train)
df_preprocessed_texts_all=pd.concat([df_preprocessed_texts_test,df_preprocessed_texts_train]).sort_index()
words_to_ints = pd.read_pickle(os.path.join(models_folder_name_train, 'doc2vec_recipes_dict_words_integers.pkl'))

preprocessed_texts_test = df_preprocessed_texts_test.preprocessed_texts.values.tolist()
preprocessed_texts_train = df_preprocessed_texts_train.preprocessed_texts.values.tolist()
preprocessed_texts_all = df_preprocessed_texts_all.preprocessed_texts.values.tolist()
labels = df_preprocessed_texts_train['labels'].values.tolist()

unique_labels=sorted(set(labels))
number_categories=len(unique_labels)
categories_indices=np.linspace(0,number_categories-1,number_categories,dtype=int)
labels2integers=dict(zip(unique_labels,categories_indices))

print(labels2integers)
df_preprocessed_texts_all

{'deviled_eggs': 6, 'macaroni_and_cheese': 15, 'sushi': 23, 'pancakes': 19, 'lasagna': 14, 'macarons': 16, 'tacos': 24, 'waffles': 26, 'hummus': 13, 'chicken_curry': 2, 'sashimi': 21, 'french_toast': 8, 'apple_pie': 0, 'frozen_yogurt': 10, 'tiramisu': 25, 'steak': 22, 'chicken_wings': 3, 'chocolate_cake': 4, 'chocolate_mousse': 5, 'garlic_bread': 11, 'fried_rice': 9, 'cheesecake': 1, 'hamburger': 12, 'onion_rings': 18, 'pizza': 20, 'dumplings': 7, 'nachos': 17}


,labels,number_of_important_words,preprocessed_texts,text_names
0,apple_pie,58,"[apple, pie, recipe, dessert, cinnamon, roll, ...",1.txt
1,apple_pie,66,"[apple, pie, dessert, apple, powder, butter, s...",10.txt
2,apple_pie,28,"[apple, pie, dessert, puff, pastry, butter, su...",11.txt
3,apple_pie,78,"[caramel, apple, pie, recipe, dessert, granny,...",12.txt
4,apple_pie,28,"[cooker, apple, recipe, dessert, apple, pie, f...",13.txt
5,apple_pie,46,"[cooker, cranberry, apple, pie, recipe, desser...",14.txt
6,apple_pie,82,"[apple, pie, recipe, dessert, flour, sugar, sa...",15.txt
7,apple_pie,60,"[apple, pie, bread, recipe, dessert, bread, eg...",2.txt
8,apple_pie,69,"[apple, pie, recipe, bakery, pie, crust, cream...",3.txt
9,apple_pie,85,"[apple, pie, recipe, bakery, vanilla, ice, cre...",4.txt


In [3]:
batch_size = 8
generations = 200000
model_learning_rate = 0.0005

embedding_size = 27   #word embedding size
doc_embedding_size = 27  #document embedding size
concatenated_size = embedding_size + doc_embedding_size

save_embeddings_every = 5000
print_valid_every = 5000
print_loss_every = 50

In [4]:
#replace each word in texts with integer value
def text_to_numbers(preprocessed_texts, word_dict):
    data = []
    for prepr_text in preprocessed_texts:
        text_data = []
        for word in prepr_text:
            if word in word_dict:
                word_ix = word_dict[word]
                text_data.append(word_ix)
        data.append(text_data)
    return (data)


def create_batch_data(text_with_words_conv_to_numbers, batch_size=batch_size):
    batch_data = []
    label_data = []
    
    rand_text_ix = int(np.random.choice(len(text_with_words_conv_to_numbers), size=1))
    rand_text = text_with_words_conv_to_numbers[rand_text_ix]
    word_to_predict_label=np.random.choice(list(set(rand_text)), size=1,replace=False)
    
    while len(batch_data) < batch_size:
        item_in_batch=[]        
        
        label_words=np.random.choice(rand_text, size=1,replace=False)

        item_in_batch.extend(word_to_predict_label)
        item_in_batch.append(rand_text_ix)     
        label_data.extend(label_words)
        batch_data.append(item_in_batch)

        
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array(label_data))

    return (batch_data, label_data)

In [5]:
word_dictionary=words_to_ints
vocabulary_size=len(word_dictionary)
print(word_dictionary)
print(vocabulary_size)

word_dictionary_rev = dict(zip(word_dictionary.values(), word_dictionary.keys()))

{'springform': 386, 'cumin': 121, 'sugar': 401, 'cardamom': 70, 'camembert': 68, 'none': 261, 'sprinkle': 387, 'saucepan': 349, 'coating': 94, 'cajun': 66, 'heart': 193, 'round': 338, 'shrimp': 361, 'facing': 151, 'cauliflower': 73, 'work': 452, 'block': 41, 'peanut': 287, 'decker': 129, 'box': 48, 'baste': 24, 'ground': 186, 'foil': 162, 'pizza': 300, 'pesto': 291, 'handful': 191, 'sea': 352, 'air': 0, 'cling': 91, 'mat': 236, 'mini': 248, 'lasagna': 210, 'cocoa': 95, 'sieve': 362, 'packet': 272, 'pot': 307, 'truffle': 427, 'wing': 448, 'seed': 356, 'wrap': 454, 'cornstarch': 112, 'seasoning': 354, 'mesh': 243, 'spatula': 377, 'breast': 52, 'chive': 83, 'herb': 194, 'speed': 378, 'macaron': 224, 'baking': 17, 'tuna': 428, 'roll': 337, 'pipe': 298, 'coloring': 99, 'confectioner': 102, 'cone': 101, 'skewer': 366, 'base': 22, 'frosting': 167, 'area': 8, 'blend': 39, 'cider': 87, 'salmon': 343, 'chickpea': 80, 'salt': 345, 'salsa': 344, 'fryer': 169, 'fusion': 170, 'core': 108, 'bun': 60,

In [6]:
text_data_test = text_to_numbers(preprocessed_texts_test, word_dictionary)
text_data_train = text_to_numbers(preprocessed_texts_train, word_dictionary)
print(np.shape(text_data_test))
print(np.shape(text_data_train))
text_data = []
text_data.extend(text_data_train)
text_data.extend(text_data_test)
print(np.shape(text_data))
print(text_data)

(52,)
(264,)
(316,)
[[6, 293, 131, 6, 310, 63, 401, 89, 345, 282, 142, 275, 6, 63, 401, 89, 265, 345, 358, 319, 282, 142, 142, 275, 289, 6, 441, 310, 310, 441, 251, 63, 401, 89, 265, 345, 445, 251, 6, 394, 282, 312, 282, 331, 6, 251, 331, 441, 331, 6, 331, 331, 154, 445, 142, 293, 358, 293, 15, 55, 331, 275, 331, 6, 293, 145], [6, 293, 131, 319, 282, 63, 401, 6, 89, 186, 358, 282, 63, 401, 6, 249, 319, 282, 370, 6, 63, 282, 401, 282, 249, 6, 282, 145], [69, 6, 293, 330, 131, 181, 371, 6, 207, 161, 401, 186, 89, 186, 293, 119, 142, 116, 55, 401, 63, 181, 371, 6, 207, 401, 186, 89, 186, 293, 119, 142, 116, 63, 345, 312, 271, 6, 370, 207, 161, 401, 89, 6, 293, 119, 293, 6, 370, 47, 119, 293, 119, 6, 370, 135, 119, 293, 142, 293, 119, 55, 282, 349, 116, 401, 401, 445, 63, 44, 69, 69, 348, 107, 293, 345, 370, 145], [104, 115, 6, 293, 330, 131, 115, 348, 6, 293, 154, 67, 249, 63, 432, 198, 116, 115, 6, 293, 67, 249, 63, 432, 198, 116, 115, 348, 6, 293, 104, 47, 67, 249, 104, 6, 115, 370, 63,

In [7]:
valid_words = ['tuna', 'rice', 'sushi', 'roll', 'sashimi','steak','grill', 'sauce', 'cream',
               'cheesecake', 'pizza', 'lasagna', 'hamburger']

valid_examples = [word_dictionary[x] for x in valid_words if x in word_dictionary.keys()]
print(valid_examples)

[428, 333, 403, 337, 347, 391, 185, 348, 116, 77, 300, 210, 190]


In [8]:
batch_data, label_data = create_batch_data(preprocessed_texts_all)
print(batch_data)
print(label_data)
print(np.shape(label_data))

[['soy' '102']
 ['soy' '102']
 ['soy' '102']
 ['soy' '102']
 ['soy' '102']
 ['soy' '102']
 ['soy' '102']
 ['soy' '102']]
['egg' 'rice' 'egg' 'salt' 'rice' 'soy' 'sauce' 'salt']
(8,)


In [9]:
tf.reset_default_graph()
embeddings = tf.get_variable("embeddings", shape=[vocabulary_size, embedding_size], trainable=False)
doc_embeddings = tf.get_variable("doc_embeddings", shape=[len(preprocessed_texts_train), doc_embedding_size])
decoder_weights = tf.get_variable("decoder_weights", shape=[vocabulary_size, concatenated_size], trainable=False)
decoder_biases = tf.get_variable("decoder_biases", shape=[vocabulary_size], trainable=False)
print(embeddings.trainable)
restorer = tf.train.Saver(name="restoring")


False


In [10]:
sess = tf.InteractiveSession()

In [11]:
restorer.restore(sess, os.path.join(models_folder_name_train,"doc2vec_recipes_checkpoint.ckpt"))
print("Model restored.")
# Check the values of the variables
print("embeddings : %s" % embeddings.eval())
print("doc_embeddings : %s" % doc_embeddings.eval())
print("decoder_weights : %s" % decoder_weights.eval())
print("decoder_biases : %s" % decoder_biases.eval())

INFO:tensorflow:Restoring parameters from /notebooks/school/text_feature_extraction/models/train/10/doc2vec_recipes_checkpoint.ckpt
Model restored.
embeddings : [[-0.6765208  -0.02028374  0.6749721  ...  0.73815125  0.56695515
   0.03896838]
 [ 0.00754435 -0.12381801 -0.03212944 ...  0.36472398  0.85472256
  -0.8338148 ]
 [-0.00252698  0.47475803 -0.46337193 ... -0.7189209   0.7055618
   0.18893655]
 ...
 [ 0.4988878  -0.00134857 -0.6899472  ... -0.14175357  0.5502593
  -0.71772   ]
 [ 0.00183253  0.3980746  -0.37587786 ... -0.5823257   0.99532753
   0.33312145]
 [ 0.30322266 -0.00235443  0.07464378 ...  0.85145634 -0.83651346
   0.24367917]]
doc_embeddings : [[-0.24955428 -0.9826422   1.2733805  ...  0.2514961   0.42668948
  -1.8259071 ]
 [-0.35951674 -1.9170965   0.14980748 ...  1.2531807   0.7351144
  -0.81338286]
 [-1.8282828  -1.1303542   0.2403738  ...  0.18064134 -0.37028372
  -0.57124025]
 ...
 [ 0.06142383 -0.4224579  -0.71129364 ...  0.6132714  -0.39184216
   1.6800978 ]
 [-1

In [13]:
print('Creating Model')

doc_embeddings_test_initial_tensor = tf.random_uniform([len(preprocessed_texts_test), doc_embedding_size], -1.0, 1.0)
doc_embeddings_all_initial_tensor=tf.concat([doc_embeddings, doc_embeddings_test_initial_tensor],0,"concat")
doc_embeddings_all=tf.Variable(doc_embeddings_all_initial_tensor,name = "doc_embeddings_all")
sess.run(tf.initialize_variables([doc_embeddings_all]))

print(doc_embeddings.shape)
print(doc_embeddings_all.shape)

x_inputs = tf.placeholder(tf.int32, shape=[None, 2]) #2:  1 for word index and 1 for doc index
y_target = tf.placeholder(tf.int32, shape=[batch_size])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

embed= tf.nn.embedding_lookup(embeddings, x_inputs[:, 0])
    
doc_indices = tf.slice(x_inputs, [0,1],[batch_size,1])
doc_embed = tf.nn.embedding_lookup(doc_embeddings_all,doc_indices)
final_embed = tf.concat([embed, tf.squeeze(doc_embed)],1)

logits = tf.matmul(final_embed, tf.transpose(decoder_weights)) + decoder_biases

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y_target))
optimizer = tf.train.AdamOptimizer(learning_rate=model_learning_rate)
train_step = optimizer.minimize(loss)

#cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True, name="cosine_similarity")

with tf.name_scope("performance"):
    loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary')
    loss_summary = tf.summary.scalar('loss', loss_ph)
performance_summaries = tf.summary.merge([loss_summary])

saver = tf.train.Saver()
summ_writer = tf.summary.FileWriter(summaries_folder_name, sess.graph)

sess.run(tf.variables_initializer(optimizer.variables()))



print('Starting Training')


for i in range(generations):
    batch_inputs, batch_labels = create_batch_data(text_data)
    feed_dict = {x_inputs : batch_inputs, y_target : batch_labels}

    sess.run(train_step, feed_dict=feed_dict)

    if (i+1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        summ = sess.run(performance_summaries, feed_dict={loss_ph:loss_val})
        summ_writer.add_summary(summ, i+1)
        print('Loss at step {} : {}'.format(i+1, loss_val))
        
    if (i+1) % print_valid_every == 0:
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = word_dictionary_rev[valid_examples[j]]
            top_k = 5 # number of nearest neighbors
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = "Nearest to {}:".format(valid_word)
            for k in range(top_k):
                close_word = word_dictionary_rev[nearest[k]]
                log_str = '{} {},'.format(log_str, close_word)
            print(log_str)
            
    #save dictionary + embeddings
    if (i+1) % save_embeddings_every == 0:
        #save vocabulary
        with open(os.path.join(models_folder_name_test,'doc2vec_recipes_dict_words_integers.pkl'), 'wb') as f:
            pickle.dump(word_dictionary, f)
        
        #save embeddings
        model_checkpoint_path = os.path.join(os.getcwd(),models_folder_name_test,'doc2vec_recipes_checkpoint.ckpt')
        save_path = saver.save(sess, model_checkpoint_path)
        print('Model saved in file: {}'.format(save_path))
        
sess.close()

Creating Model
Instructions for updating:
Use `tf.variables_initializer` instead.
(264, 27)
(316, 27)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Starting Training
Loss at step 50 : 3.0290775299072266
Loss at step 100 : 3.360969066619873
Loss at step 150 : 8.807123184204102
Loss at step 200 : 2.406243324279785
Loss at step 250 : 3.0291762351989746
Loss at step 300 : 5.334019660949707
Loss at step 350 : 7.515295028686523
Loss at step 400 : 3.528195381164551
Loss at step 450 : 2.8757855892181396
Loss at step 500 : 6.138593673706055
Loss at step 550 : 3.0525684356689453
Loss at step 600 : 7.907103061676025
Loss at step 650 : 3.4754695892333984
Loss at step 700 : 7.502150535583496
Loss at step 750 : 6.156990051269531
Loss at step 800 : 3.054830312728882
Loss at step 850 : 3.2054712772369385
Loss at step 900 : 3.0558252334594727
Loss at step 950 : 2.8244338035583496
Loss at step 1000 : 3.4065909385681152
Loss at step 1050 : 7.052783489227295
Loss at step 1100 : 

Loss at step 8900 : 2.5960659980773926
Loss at step 8950 : 3.4089841842651367
Loss at step 9000 : 2.7690718173980713
Loss at step 9050 : 5.533332347869873
Loss at step 9100 : 2.7925872802734375
Loss at step 9150 : 2.6319339275360107
Loss at step 9200 : 3.0769405364990234
Loss at step 9250 : 3.003282070159912
Loss at step 9300 : 2.483160972595215
Loss at step 9350 : 2.9699389934539795
Loss at step 9400 : 3.6949410438537598
Loss at step 9450 : 2.8764729499816895
Loss at step 9500 : 4.223791122436523
Loss at step 9550 : 2.651318311691284
Loss at step 9600 : 2.777953624725342
Loss at step 9650 : 2.885868549346924
Loss at step 9700 : 2.435645580291748
Loss at step 9750 : 3.0214080810546875
Loss at step 9800 : 2.98506236076355
Loss at step 9850 : 7.900766372680664
Loss at step 9900 : 6.4893035888671875
Loss at step 9950 : 2.382174253463745
Loss at step 10000 : 3.262024402618408
Nearest to tuna: parmesan, metal, eye, position, mat,
Nearest to rice: thigh, lime, mandu, simmer, spinach,
Nearest

Loss at step 17400 : 3.326770782470703
Loss at step 17450 : 3.112553596496582
Loss at step 17500 : 2.4997687339782715
Loss at step 17550 : 3.1086995601654053
Loss at step 17600 : 3.01926326751709
Loss at step 17650 : 2.7939977645874023
Loss at step 17700 : 2.3297345638275146
Loss at step 17750 : 3.2448205947875977
Loss at step 17800 : 3.6008403301239014
Loss at step 17850 : 1.8896619081497192
Loss at step 17900 : 4.087776184082031
Loss at step 17950 : 2.192676305770874
Loss at step 18000 : 2.0449037551879883
Loss at step 18050 : 3.2113659381866455
Loss at step 18100 : 5.101892471313477
Loss at step 18150 : 3.6930034160614014
Loss at step 18200 : 2.5765442848205566
Loss at step 18250 : 2.412639617919922
Loss at step 18300 : 5.222782135009766
Loss at step 18350 : 4.448805809020996
Loss at step 18400 : 4.027799606323242
Loss at step 18450 : 2.8745079040527344
Loss at step 18500 : 2.7000396251678467
Loss at step 18550 : 2.1796514987945557
Loss at step 18600 : 2.349194288253784
Loss at step

Loss at step 25700 : 3.039734363555908
Loss at step 25750 : 2.6663432121276855
Loss at step 25800 : 2.7856807708740234
Loss at step 25850 : 2.611057996749878
Loss at step 25900 : 2.8066418170928955
Loss at step 25950 : 2.537956714630127
Loss at step 26000 : 2.3649845123291016
Loss at step 26050 : 3.2953107357025146
Loss at step 26100 : 3.114234685897827
Loss at step 26150 : 2.3381738662719727
Loss at step 26200 : 4.456948280334473
Loss at step 26250 : 2.3253114223480225
Loss at step 26300 : 4.396428108215332
Loss at step 26350 : 3.253453493118286
Loss at step 26400 : 3.1802384853363037
Loss at step 26450 : 3.8754372596740723
Loss at step 26500 : 3.005016803741455
Loss at step 26550 : 3.6911137104034424
Loss at step 26600 : 3.646693706512451
Loss at step 26650 : 3.5391111373901367
Loss at step 26700 : 2.095207929611206
Loss at step 26750 : 3.050137996673584
Loss at step 26800 : 3.7426345348358154
Loss at step 26850 : 2.600449323654175
Loss at step 26900 : 2.1537070274353027
Loss at step

Loss at step 35100 : 3.294724941253662
Loss at step 35150 : 3.949187755584717
Loss at step 35200 : 2.3460521697998047
Loss at step 35250 : 2.9438014030456543
Loss at step 35300 : 3.053992748260498
Loss at step 35350 : 2.3246192932128906
Loss at step 35400 : 3.795971393585205
Loss at step 35450 : 2.927128791809082
Loss at step 35500 : 3.286423921585083
Loss at step 35550 : 3.9401941299438477
Loss at step 35600 : 2.8243870735168457
Loss at step 35650 : 5.784549236297607
Loss at step 35700 : 3.1891446113586426
Loss at step 35750 : 2.960413694381714
Loss at step 35800 : 3.191460132598877
Loss at step 35850 : 3.498077869415283
Loss at step 35900 : 2.4937398433685303
Loss at step 35950 : 3.155271053314209
Loss at step 36000 : 4.75688362121582
Loss at step 36050 : 1.8074861764907837
Loss at step 36100 : 2.6644864082336426
Loss at step 36150 : 2.779261827468872
Loss at step 36200 : 2.887299060821533
Loss at step 36250 : 2.036921501159668
Loss at step 36300 : 3.0575056076049805
Loss at step 363

...

Loss at step 164950 : 3.106454849243164
Loss at step 165000 : 3.5526018142700195
Nearest to tuna: parmesan, metal, eye, position, mat,
Nearest to rice: thigh, lime, mandu, simmer, spinach,
Nearest to sushi: piece, sashimi, sesame, rare, bundt,
Nearest to roll: meal, sheet, ricotta, salt, oil,
Nearest to sashimi: fryer, seaweed, sushi, ring, gyoza,
Nearest to steak: maple, grate, eye, meat, marinade,
Nearest to grill: soy, spring, teriyaki, steak, cider,
Nearest to sauce: appetizer, jack, wasabi, oregano, soy,
Nearest to cream: chocolate, cherry, extract, cake, oven,
Nearest to cheesecake: cashew, nutmeg, flake, tin, chill,
Nearest to pizza: flake, white, baguette, mixture, run,
Nearest to lasagna: fry, noodle, rare, lemon, teriyaki,
Nearest to hamburger: herb, meat, pine, tomato, ½,
Model saved in file: /notebooks/school/text_feature_extraction/models/test/10/doc2vec_recipes_checkpoint.ckpt
Loss at step 165050 : 2.114621162414551
Loss at step 165100 : 2.7790424823760986
Loss at step 16

Loss at step 173050 : 2.7165729999542236
Loss at step 173100 : 2.7066116333007812
Loss at step 173150 : 4.08202600479126
Loss at step 173200 : 3.42217755317688
Loss at step 173250 : 2.8906545639038086
Loss at step 173300 : 2.244373083114624
Loss at step 173350 : 2.066524028778076
Loss at step 173400 : 2.4893829822540283
Loss at step 173450 : 2.742922306060791
Loss at step 173500 : 3.0356266498565674
Loss at step 173550 : 3.0591681003570557
Loss at step 173600 : 2.757065534591675
Loss at step 173650 : 3.519181728363037
Loss at step 173700 : 2.618668556213379
Loss at step 173750 : 2.9896864891052246
Loss at step 173800 : 1.9236602783203125
Loss at step 173850 : 2.8907907009124756
Loss at step 173900 : 2.618011713027954
Loss at step 173950 : 3.4663078784942627
Loss at step 174000 : 4.155165672302246
Loss at step 174050 : 2.917320489883423
Loss at step 174100 : 3.1401357650756836
Loss at step 174150 : 2.511472702026367
Loss at step 174200 : 3.7290937900543213
Loss at step 174250 : 2.154455

Loss at step 181150 : 2.585817337036133
Loss at step 181200 : 2.3295116424560547
Loss at step 181250 : 2.430349349975586
Loss at step 181300 : 2.091128349304199
Loss at step 181350 : 2.454464912414551
Loss at step 181400 : 2.41652250289917
Loss at step 181450 : 2.7743120193481445
Loss at step 181500 : 3.350148916244507
Loss at step 181550 : 3.999124526977539
Loss at step 181600 : 3.0804367065429688
Loss at step 181650 : 3.120558738708496
Loss at step 181700 : 3.770890712738037
Loss at step 181750 : 3.6661205291748047
Loss at step 181800 : 2.3058876991271973
Loss at step 181850 : 3.052584171295166
Loss at step 181900 : 3.568582534790039
Loss at step 181950 : 3.1105847358703613
Loss at step 182000 : 3.1838536262512207
Loss at step 182050 : 3.504340887069702
Loss at step 182100 : 2.2642781734466553
Loss at step 182150 : 3.7928175926208496
Loss at step 182200 : 3.5463552474975586
Loss at step 182250 : 3.6363449096679688
Loss at step 182300 : 3.0566935539245605
Loss at step 182350 : 3.36619

Loss at step 190050 : 3.4357147216796875
Loss at step 190100 : 1.83039128780365
Loss at step 190150 : 3.575221538543701
Loss at step 190200 : 3.0421972274780273
Loss at step 190250 : 2.6529839038848877
Loss at step 190300 : 2.8612523078918457
Loss at step 190350 : 2.809826135635376
Loss at step 190400 : 3.0876691341400146
Loss at step 190450 : 3.0990548133850098
Loss at step 190500 : 3.336799383163452
Loss at step 190550 : 2.760427474975586
Loss at step 190600 : 3.204167366027832
Loss at step 190650 : 2.8181262016296387
Loss at step 190700 : 3.9046783447265625
Loss at step 190750 : 3.3070478439331055
Loss at step 190800 : 2.8177614212036133
Loss at step 190850 : 3.0169589519500732
Loss at step 190900 : 3.211078643798828
Loss at step 190950 : 2.4265642166137695
Loss at step 191000 : 3.499279737472534
Loss at step 191050 : 2.478121757507324
Loss at step 191100 : 2.7809319496154785
Loss at step 191150 : 2.956204414367676
Loss at step 191200 : 2.1574721336364746
Loss at step 191250 : 3.653

Loss at step 199250 : 3.577129364013672
Loss at step 199300 : 3.6276893615722656
Loss at step 199350 : 2.6090211868286133
Loss at step 199400 : 2.7248284816741943
Loss at step 199450 : 3.24519681930542
Loss at step 199500 : 2.1846117973327637
Loss at step 199550 : 2.3045291900634766
Loss at step 199600 : 2.9513297080993652
Loss at step 199650 : 3.098628520965576
Loss at step 199700 : 3.2195024490356445
Loss at step 199750 : 2.6985244750976562
Loss at step 199800 : 3.075005292892456
Loss at step 199850 : 2.0871458053588867
Loss at step 199900 : 3.2238855361938477
Loss at step 199950 : 2.687626838684082
Loss at step 200000 : 2.971001386642456
Nearest to tuna: parmesan, metal, eye, position, mat,
Nearest to rice: thigh, lime, mandu, simmer, spinach,
Nearest to sushi: piece, sashimi, sesame, rare, bundt,
Nearest to roll: meal, sheet, ricotta, salt, oil,
Nearest to sashimi: fryer, seaweed, sushi, ring, gyoza,
Nearest to steak: maple, grate, eye, meat, marinade,
Nearest to grill: soy, spring